# Nando CSPC 540

[website](http://www.cs.ubc.ca/~nando/540-2013/lectures.html)

## Gaussian Process

[Video](https://www.youtube.com/watch?v=4vGiHC35j9s)

* Vector dot products measure similarity

For covariance matrices:

$$
\begin{aligned}
std(x) = \sigma(x_1) = \sqrt{\mathbb{E}\big(x_1^2\big)} \\
covariance(x_1,x_2) = \mathbb{E}\big(x_1 x_2\big) \\
\end{aligned}
$$

For a 2-d joint Gaussian distributions, $X_1$ and $X_2$, the 1-d slice given $X_1 = x_1$ is:

$$ P(X_2 \mid X_1 = x_1)$$

If we have the data for the 2-d distribution, to arrive at the 1-d distrubtion, **read Chapter 4 Kevin Murphy's book on machine learning**. He provided the derivation. This allows you to go from the joint distribution to a single distribution.

### Sampling, how to sample from Gaussian

To sample an univariate Gaussian distributions, draw from a uniform distribution / random number generator, project onto a cumulative Gaussian probability function, and then project to a probability density function. Video / slides at 29:00.

This can generate a standard Gaussian of $\mathcal{N}(0, 1)$.

**Square root of matrices**: **Cholesky** decomposition of matrix $\Sigma$:
$$ \Sigma = LL^T $$

This allows us to sample multivariate Gaussian distributions. Using Multivariate Gaussian Theorem.

Covariances can be modelled as similarity, e.g. $K_{ij}$ is a value in the covariance matrix:

$$K_{ij} = \exp\big({-\left\Vert x_i - x_j \right\Vert^2}\big) $$

This is a Euclidean kernel.